In [56]:
import pandas as pd
import requests
import numpy as np
import yaml
import json
import time

## NBA Data Pull

In [22]:
r = requests.get('https://www.balldontlie.io/api/v1/players', params={'per_page': 100})
j = json.loads(r.content.decode())
pd.DataFrame(j['data'])

r_list = [requests.get('https://www.balldontlie.io/api/v1/players', params={'per_page': 100, 'page': n}) for n in range(0, j['meta']['total_pages'])]

df_players = pd.concat([pd.json_normalize(json.loads(m.content.decode())['data']) for m in r_list])
df_players = df_players.sort_values('id').drop_duplicates().set_index('id')

df_players.to_csv('player_list.csv')

In [49]:
r = requests.get('https://www.balldontlie.io/api/v1/teams', params={'per_page': 100})
j = json.loads(r.content.decode())
pd.DataFrame(j['data'])

r_list = [requests.get('https://www.balldontlie.io/api/v1/teams', params={'per_page': 100, 'page': n}) for n in range(0, j['meta']['total_pages'])]

df_teams = pd.concat([pd.json_normalize(json.loads(m.content.decode())['data']) for m in r_list])
df_teams = df_teams.sort_values('id').drop_duplicates().set_index('id')

df_teams.to_csv('team_list.csv')

In [ ]:
r = requests.get('https://www.balldontlie.io/api/v1/games', params={'per_page': 100})
j = json.loads(r.content.decode())
pd.DataFrame(j['data'])

#rate limit hecks you up here if you don't pace urself

r_list = []

for i in range(0, j['meta']['total_pages']):
    time.sleep(1)
    r_list.append(
        requests.get('https://www.balldontlie.io/api/v1/games',
                     params={'per_page': 100, 'page': i}
                     )
    )
#df_teams = pd.concat([pd.json_normalize(json.loads(m.content.decode())['data']) for m in r_list])
#df_teams = df_teams.sort_values('id').drop_duplicates().set_index('id')

#df_teams.to_csv('team_list.csv')

In [ ]:
df_games = pd.concat([pd.json_normalize(json.loads(m.content.decode())['data']) for m in r_list])
#df_games = df_games.sort_values('id').drop_duplicates().set_index('id')

df_games
# df_games.to_csv('game_list.csv')